In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./dataset/train',
                                                    target_size=(48,48),
                                                    batch_size=64,
                                                    color_mode='grayscale',
                                                    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory('./dataset/test',
                                                      target_size=(48,48),
                                                      batch_size=64,
                                                      color_mode='grayscale',
                                                      class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.fit_generator(train_generator,
                   steps_per_epoch=28709//64,
                   epochs=30,
                   validation_data = validation_generator,
                   validation_steps=7178//64)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 448 steps, validate for 112 steps
Epoch 1/30
448/448 [==============================] - 184s 411ms/step - loss: 1.7446 - accuracy: 0.2865 - val_loss: 1.5627 - val_accuracy: 0.3792
Epoch 2/30
448/448 [==============================] - 199s 445ms/step - loss: 1.4358 - accuracy: 0.4435 - val_loss: 1.3571 - val_accuracy: 0.4693
Epoch 3/30
448/448 [==============================] - 197s 439ms/step - loss: 1.2859 - accuracy: 0.5082 - val_loss: 1.2153 - val_accuracy: 0.5280
Epoch 4/30
448/448 [==============================] - 202s 450ms/step - loss: 1.2052 - accuracy: 0.5402 - val_loss: 1.1653 - val_accuracy: 0.5532
Epoch 5/30
448/448 [==============================] - 200s 447ms/step - loss: 1.1542 - accuracy: 0.5641 - val_loss: 1.1319 - val_accuracy: 0.5636
Epoch 6/30
448/448 [==============================] - 200s 445ms/step - loss: 1.1108 - accuracy: 0.5

In [6]:
model.save('model.h5')

In [7]:
train_generator.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [12]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import numpy as np
model = load_model('model.h5')
result = {0:'angry',
 1:'disgust',
 2:'fear',
 3:'happy',
 4:'neutral',
 5:'sad',
 6:'surprise'}

In [23]:
test_image = './dataset/test_data/happy.jpg'

In [24]:
img = image.load_img(test_image, target_size=(48,48))
img = img.convert('L')
x = image.img_to_array(img)
x /= 255
x = x.reshape(1, 48, 48, 1)

In [25]:
predict = model.predict(x)
print(predict)

[[8.4758598e-01 1.1628515e-12 2.5089108e-03 4.8195983e-05 1.4401911e-01
  5.8035548e-03 3.4268760e-05]]


In [26]:
final_prediction = result[np.argmax(predict[0])]

In [27]:
print(final_prediction)

angry
